In [1]:
import os
import logging
import argparse
import pandas as pd
import numpy as np
import pytz
import pyarrow
from pathlib import Path
from transformers import BertTokenizer, BertModel, BertConfig, BertForTokenClassification, pipeline, \
    AutoModelForTokenClassification, AutoTokenizer
import torch
import nltk
from nltk.corpus import stopwords
import string
from nltk.util import skipgrams
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from collections import Counter

In [2]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
               'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated","repeated",
              'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens

    # corpus from which the word statistics are going to be used
    # for word segmentation
    segmenter="twitter",

    # corpus from which the word statistics are going to be used
    # for spell correction
    corrector="twitter",

    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


/home/manuto/.local/lib/python3.8/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/home/manuto/.local/lib/python3.8/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [3]:
def k_skip_n_grams(sent, k, n):
    return list(skipgrams(sent, k=k, n=n))

In [22]:
df = pd.read_csv("/home/manuto/Documents/world_bank/bert_twitter_labor/twitter-labor-data/data/jul23_iter0/preprocessed/train_is_unemployed.csv", lineterminator='\n')
df.head()

,tweet_id,text,class
0,524448969363963905,I only work 4 days next week which is fine but...,0.0
1,516983754045526016,@Lawwren__ We saw that you need a job we would...,0.0
2,440127142966210560,me: if I eat food in the kitchen will I get la...,0.0
3,557797964396388352,Currently curled in fetal position because of ...,0.0
4,412754561527316480,I am currently texting my nephews ex girlfrien...,0.0


In [23]:
df['tokenized_preprocessed_text'] = df['text'].apply(text_processor.pre_process_doc)
df.head()

,tweet_id,text,class,tokenized_preprocessed_text
0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh..."
1,516983754045526016,@Lawwren__ We saw that you need a job we would...,0.0,"[<user>, we, saw, that, you, need, a, job, we,..."
2,440127142966210560,me: if I eat food in the kitchen will I get la...,0.0,"[me, :, if, i, eat, food, in, the, kitchen, wi..."
3,557797964396388352,Currently curled in fetal position because of ...,0.0,"[currently, curled, in, fetal, position, becau..."
4,412754561527316480,I am currently texting my nephews ex girlfrien...,0.0,"[i, am, currently, texting, my, nephews, ex, g..."


In [24]:
df1 = df
df1['skipgrams'] = df1['tokenized_preprocessed_text'].apply(k_skip_n_grams,k=2,n=3)
df1.head()

,tweet_id,text,class,tokenized_preprocessed_text,skipgrams
0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","[(i, only, work), (i, only, <number>), (i, onl..."
1,516983754045526016,@Lawwren__ We saw that you need a job we would...,0.0,"[<user>, we, saw, that, you, need, a, job, we,...","[(<user>, we, saw), (<user>, we, that), (<user..."
2,440127142966210560,me: if I eat food in the kitchen will I get la...,0.0,"[me, :, if, i, eat, food, in, the, kitchen, wi...","[(me, :, if), (me, :, i), (me, :, eat), (me, i..."
3,557797964396388352,Currently curled in fetal position because of ...,0.0,"[currently, curled, in, fetal, position, becau...","[(currently, curled, in), (currently, curled, ..."
4,412754561527316480,I am currently texting my nephews ex girlfrien...,0.0,"[i, am, currently, texting, my, nephews, ex, g...","[(i, am, currently), (i, am, texting), (i, am,..."


In [10]:
df['checkisin'] = [top_structure in structure_list for top_structure, structure_list in zip([('i','only','work')]*df.shape[0], df['skipgrams'])]
df.head()

,index,tweet_id,text,class,tokenized_preprocessed_text,skipgrams,checkisin
0,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","[(i, only, work), (i, only, <number>), (i, onl...",True
1,1,516983754045526016,@Lawwren__ We saw that you need a job we would...,0.0,"[<user>, we, saw, that, you, need, a, job, we,...","[(<user>, we, saw), (<user>, we, that), (<user...",False
2,2,440127142966210560,me: if I eat food in the kitchen will I get la...,0.0,"[me, :, if, i, eat, food, in, the, kitchen, wi...","[(me, :, if), (me, :, i), (me, :, eat), (me, i...",False
3,3,557797964396388352,Currently curled in fetal position because of ...,0.0,"[currently, curled, in, fetal, position, becau...","[(currently, curled, in), (currently, curled, ...",False
4,4,412754561527316480,I am currently texting my nephews ex girlfrien...,0.0,"[i, am, currently, texting, my, nephews, ex, g...","[(i, am, currently), (i, am, texting), (i, am,...",False


In [19]:
df = df[df['checkisin']]
df.head()

,index,tweet_id,text,class,tokenized_preprocessed_text,skipgrams,checkisin
0,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","[(i, only, work), (i, only, <number>), (i, onl...",True


In [11]:
skipgrams_count = df.explode('skipgrams').reset_index(drop=True)
skipgrams_count.head()

,index,tweet_id,text,class,tokenized_preprocessed_text,skipgrams,checkisin
0,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, work)",True
1,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, <number>)",True
2,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, days)",True
3,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, work, <number>)",True
4,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, work, days)",True


In [12]:
skipgrams_count['skipgrams'][0]

('i', 'only', 'work')

In [13]:
skipgrams_count['share_specific_tokens'] = skipgrams_count['skipgrams'].apply(lambda token_list: sum('<' in token for token in [str(i) for i in token_list])/len(token_list))
punctuation_list = [i for i in string.punctuation]
skipgrams_count['share_punctuation'] = skipgrams_count['skipgrams'].apply(lambda token_list: len(list(set(token_list).intersection(punctuation_list))) / len(token_list))
skipgrams_count['total_share_irrelevant_tokens'] = skipgrams_count['share_specific_tokens'] + skipgrams_count['share_punctuation']
skipgrams_count.head(n=50)

,index,tweet_id,text,class,tokenized_preprocessed_text,skipgrams,checkisin,share_specific_tokens,share_punctuation,total_share_irrelevant_tokens
0,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, work)",True,0.000000,0.0,0.000000
1,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, <number>)",True,0.333333,0.0,0.333333
2,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, days)",True,0.000000,0.0,0.000000
3,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, work, <number>)",True,0.333333,0.0,0.333333
4,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, work, days)",True,0.000000,0.0,0.000000
5,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, <number>, days)",True,0.333333,0.0,0.333333
6,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(only, work, <number>)",True,0.333333,0.0,0.333333
7,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(only, work, days)",True,0.000000,0.0,0.000000
8,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(only, work, next)",True,0.000000,0.0,0.000000
9,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(only, <number>, days)",True,0.333333,0.0,0.333333


In [14]:
skipgrams_count = skipgrams_count[skipgrams_count['total_share_irrelevant_tokens']<(2/3)].reset_index(drop=True)
top_structures_dict = dict(skipgrams_count['skipgrams'].value_counts(dropna=False))
top_structures_dict

{('i', 'am', 'currently'): 150,
 ('i', 'am', 'unemployed'): 98,
 ('.', 'i', 'am'): 82,
 ('i', 'laid', 'off'): 61,
 ('i', 'am', 'a'): 53,
 ('i', 'am', 'to'): 50,
 ('i', 'am', 'not'): 49,
 ('i', 'am', '.'): 49,
 ('and', 'i', 'am'): 45,
 ('i', 'am', 'for'): 44,
 (',', 'i', 'am'): 43,
 ('i', 'quit', 'i'): 43,
 ('i', 'am', 'working'): 43,
 ('laid', 'off', '.'): 42,
 ('quit', 'i', 'quit'): 40,
 ('i', 'do', 'not'): 38,
 ('i', 'am', 'in'): 37,
 ('it', "'", 's'): 35,
 ('i', 'am', 'the'): 34,
 ('i', 'am', 'and'): 32,
 ('now', 'i', 'am'): 32,
 ('got', 'laid', 'off'): 31,
 ('i', 'am', 'now'): 30,
 ('was', 'laid', 'off'): 29,
 ('.', 'i', 'have'): 27,
 ('am', 'currently', '.'): 26,
 ('i', 'am', 'on'): 26,
 ('laid', 'off', '<number>'): 25,
 ('am', 'unemployed', '.'): 25,
 ('i', 'have', 'a'): 25,
 ('am', 'currently', 'working'): 24,
 ('am', 'currently', 'a'): 24,
 ('i', 'have', 'been'): 24,
 ('for', 'a', 'job'): 24,
 ('quit', 'my', 'job'): 23,
 ('i', 'was', 'off'): 23,
 ('i', 'was', 'laid'): 23,
 ('i'

In [15]:
lst2 = [item[0] for item in Counter(top_structures_dict).most_common(3)]
lst2

[('i', 'am', 'currently'), ('i', 'am', 'unemployed'), ('.', 'i', 'am')]

In [25]:
lst2[:2]

[('i', 'am', 'currently'), ('i', 'am', 'unemployed')]

In [17]:
"{}_in_skipgrams".format(lst2[0])

"('i', 'am', 'currently')_in_skipgrams"

In [18]:
skipgrams_count["{}_in_skipgrams".format(lst2[0])] = 0
skipgrams_count.head()

,index,tweet_id,text,class,tokenized_preprocessed_text,skipgrams,checkisin,share_specific_tokens,share_punctuation,total_share_irrelevant_tokens,"('i', 'am', 'currently')_in_skipgrams"
0,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, work)",True,0.000000,0.0,0.000000,0
1,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, <number>)",True,0.333333,0.0,0.333333,0
2,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, only, days)",True,0.000000,0.0,0.000000,0
3,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, work, <number>)",True,0.333333,0.0,0.333333,0
4,0,524448969363963905,I only work 4 days next week which is fine but...,0.0,"[i, only, work, <number>, days, next, week, wh...","(i, work, days)",True,0.000000,0.0,0.000000,0


In [ ]:
skipgrams_count = skipgrams_count[skipgrams_count['checkisin']]
skipgram_count